In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
class EffectType_s():
    def __init__(self, tgt, effect):
        self.tgt = tgt
        self.effect = effect
    def __repr__(self):
        return f"ET({self.tgt},{self.effect})"
#     def __eq__(self, other):
# 		return z3.eq(self.pvar, other.pvar) and self.index == other.index
# 	def __lt__(self, other):
# 		# TODO incorporate pvar type into sort.
# 		if str(self.pvar) > str(other.pvar): return False
# 		if str(self.pvar) == str(other.pvar) and self.index >= other.index: return False
# 		return True
# 	def __hash__(self):
# 		return hash((hash(self.pvar), hash(self.index)))

In [43]:
from action import Action
from PDDL import PDDL_Parser
import sys, pprint
from collections import OrderedDict
import re, copy
import itertools
import z3

**TODO**

* Construct pvars
* Create EffectType class with eq, lt, hash

In [4]:
def list_is_flat(l):
    for x in l: 
        if isinstance(x, list):
            return False
    return True

In [5]:
def make_EffectType_s(l, is_add = True):
    if list_is_flat(l):
        tgt = l
        effect = is_add
    else:
        tgt = l[1]
        effect = [l[0],l[-1]]
    return EffectType_s(tgt, effect)

In [6]:
def action2EffectTypes(act):
    return [make_EffectType_s(x,True) for x in act.add_effects] + [make_EffectType_s(x,False) for x in act.del_effects]

In [7]:
zeno_dom = "examples/zeno/zeno.pddl"
zeno_prob = "examples/zeno/pb1.pddl"

In [8]:
dinner_dom = "./examples/dinner/dinner.pddl"
dinner_prob = "./examples/dinner/pb1.pddl"

In [17]:
# domain, problem = dinner_dom, dinner_prob
domain, problem = zeno_dom, zeno_prob

parser = PDDL_Parser()
print('----------------------------')
# pprint.pprint(parser.scan_tokens(problem))
print('----------------------------')
parser.parse_domain(domain)
parser.parse_problem(problem)
print('Domain name: ' + parser.domain_name)
print(f'~~~Types~~~')
for t in parser.types: print(t)
print(f'~~~Type Hierarchy~~~')
for k,v in parser.type_hierarchy.items():
    print(f"{k}: {v}")
print(f'~~~Predicates~~~')
for nm, args in parser.predicates.items():
    print(f"{nm}({args}")
#     print("")
print('----------------------------')
print(f'~~~Functions~~~')
for nm, args in parser.functions.items():
    print(f"{nm}({args}")
#     print("")
print('----------------------------')
print('~~~Actions~~~')
for act in parser.actions:
    print(act)
    # print("effect types:")
    # for ef in action2EffectTypes(act): print(f"\t{ef}")
    print("\n")
print('----------------------------')
print('Problem name: ' + parser.problem_name)
print('Objects: ' + str(parser.objects))
print('State: ' + str(parser.state))
print('Positive goals: ' + str(parser.positive_goals))
print('Negative goals: ' + str(parser.negative_goals))

----------------------------
----------------------------
['locatable city - object', 'aircraft person - locatable']
Domain name: zenotravel
~~~Types~~~
aircraft
city
locatable
object
person
~~~Type Hierarchy~~~
aircraft: []
city: []
locatable: ['aircraft', 'person']
object: ['city', 'locatable']
person: []
~~~Predicates~~~
located(OrderedDict([('?x', 'locatable'), ('?c', 'city')])
in(OrderedDict([('?p', 'person'), ('?a', 'aircraft')])
----------------------------
~~~Functions~~~
fuel(OrderedDict([('?a', 'aircraft')])
distance(OrderedDict([('?c1', 'city'), ('?c2', 'city')])
slow-burn(OrderedDict([('?a', 'aircraft')])
fast-burn(OrderedDict([('?a', 'aircraft')])
capacity(OrderedDict([('?a', 'aircraft')])
total-fuel-used(OrderedDict()
onboard(OrderedDict([('?a', 'aircraft')])
zoom-limit(OrderedDict([('?a', 'aircraft')])
----------------------------
~~~Actions~~~
action: board
  parameters: [['?p', 'person'], ['?a', 'aircraft'], ['?c', 'city']]
  positive_preconditions: [['located', '?p', 

In [23]:
parser.types

['aircraft', 'city', 'locatable', 'object', 'person']

In [21]:
nm, var_nm_types = list(parser.predicates.items())[0]

In [22]:
print(nm)
print(var_nm_types)

located
OrderedDict([('?x', 'locatable'), ('?c', 'city')])


In [24]:
varnm2objects = OrderedDict()
for varnm, vartype in var_nm_types.items():
    subtypes = parser.get_subtypes([vartype])
    valid_objects = []
    for st in subtypes:
        if st in parser.objects.keys():
            valid_objects.extend(parser.objects[st])
    varnm2objects[varnm] = valid_objects

In [25]:
varnm2objects

OrderedDict([('?x', ['plane1', 'person1', 'person2', 'person3']),
             ('?c', ['city0', 'city1', 'city2'])])

In [40]:
# Note: the * operator here is used to unpack the values so the itertools.product function
# can operate over all of them as opposed to producting a single list (which is useless)
grounded_pvar_predicate_list = list(itertools.product(*varnm2objects.values()))
print(grounded_pvar_predicate_list)

[('plane1', 'city0'), ('plane1', 'city1'), ('plane1', 'city2'), ('person1', 'city0'), ('person1', 'city1'), ('person1', 'city2'), ('person2', 'city0'), ('person2', 'city1'), ('person2', 'city2'), ('person3', 'city0'), ('person3', 'city1'), ('person3', 'city2')]


In [44]:
predicate_z3_list = []
for arg_tuple in grounded_pvar_predicate_list:
    predicate_z3_list.append(z3.Bool(nm+str(arg_tuple)))
print(predicate_z3_list)

[located('plane1', 'city0'), located('plane1', 'city1'), located('plane1', 'city2'), located('person1', 'city0'), located('person1', 'city1'), located('person1', 'city2'), located('person2', 'city0'), located('person2', 'city1'), located('person2', 'city2'), located('person3', 'city0'), located('person3', 'city1'), located('person3', 'city2')]
